# Treasure Hunt Game Notebook

## Read and Review Your Starter Code
The theme of this project is a popular treasure hunt game in which the player needs to find the treasure before the pirate does. While you will not be developing the entire game, you will write the part of the game that represents the intelligent agent, which is a pirate in this case. The pirate will try to find the optimal path to the treasure using deep Q-learning. 

<div class="alert alert-block alert-success" style="color:black;">
<b>To Begin:</b> Use this <b>TreasureHuntGame_starterCode.ipynb</b> file to complete your assignment. 
<br><br>
You have been provided with two Python classes and this notebook to help you with this assignment. The first class, <b>TreasureMaze.py</b>, represents the environment, which includes a maze object defined as a matrix. The second class, <b>GameExperience.py</b>, stores the episodes – that is, all the states that come in between the initial state and the terminal state. This is later used by the agent for learning by experience, called "exploration". This notebook shows how to play a game. Your task is to complete the deep Q-learning implementation in the qtrain() function for which a skeleton implementation has been provided. 
</div>
<br>
<div class="alert alert-block alert-info" style="color:black;">
<b>NOTE: </b>The code block you will need to complete will have <b>#TODO</b> as a header.
<br> First, read and review the next few code and instruction blocks to understand the code that you have been given.</div>

<div class="alert alert-block alert-warning" style="color: #333333;">
<b>Installations</b> The following command will install the necessary Python libraries to necessary to run this application. If you see a "[notice] A new release of pip is available: 23.1.2 -> 25.2" at the end of the installation, you may disregard that statement. 
</div>

In [ ]:
!pip install -r requirements.txt

<h2>Tensorflow CPU Acceleration Warning</h2>
<div class="alert alert-block alert-danger" style="color: #333333;">
<b>GPU/CUDA/Memory Warnings/Errors:</b> You may receive some errors referencing that GPUs will not be used, CUDA could not be found, or free system memory allocation errors. These and a few others, are standard errors that can be ignored here as they are environment based.<br><br>
    <b>Example messages:</b>
    <ul>
        <li>oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders</li>
        <li>WARNING: All log messages before absl::InitializeLog() is called are written to STDERR</li>
</div>

In [ ]:
# Block 0: TensorFlow Configuration
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress most warnings
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN if causing issues

In [ ]:
# Block 1: Imports
from __future__ import print_function
import os, sys, time, datetime, json, random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import clone_model, Sequential
from tensorflow.keras.layers import Dense, Activation, PReLU
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt
from TreasureMaze import TreasureMaze
from GameExperience import GameExperience
%matplotlib inline

# Configure GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(f"GPU config error: {e}")

# Clear session
tf.keras.backend.clear_session()

<h2> Maze Object Generation</h2>

<div class="alert alert-block alert-info" style="color:black;">
    <b>NOTE:</b>  The following code block contains an 8x8 matrix that will be used as a maze object:
</div>

In [ ]:
# Block 2: Maze Definition
maze = np.array([
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.]
])

print("Maze shape:", maze.shape)
print("Number of free cells:", np.sum(maze == 1))

<h2>Helper Functions and Global Variables</h2>

<div class="alert alert-block alert-info" style="color:black;">
This <b>show()</b> helper function allows a visual representation of the maze object:
</div>

In [ ]:
# Block 3: Visualization Function
def show(qmaze):
    """Display the current maze state with pirate position and visited cells"""
    plt.grid('on')
    nrows, ncols = qmaze.maze.shape
    ax = plt.gca()
    ax.set_xticks(np.arange(0.5, nrows, 1))
    ax.set_yticks(np.arange(0.5, ncols, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    canvas = np.copy(qmaze.maze)
    for row,col in qmaze.visited:
        canvas[row,col] = 0.6
    pirate_row, pirate_col, _ = qmaze.state
    canvas[pirate_row, pirate_col] = 0.3   # pirate cell
    canvas[nrows-1, ncols-1] = 0.9 # treasure cell
    img = plt.imshow(canvas, interpolation='none', cmap='gray')
    return img

The <b>pirate agent</b> can move in four directions: left, right, up, and down. 

<div class="alert alert-block alert-warning" style="color:black;">
<b>Note:</b> While the agent primarily learns by experience through exploitation, often, the agent can choose to explore the environment to find previously undiscovered paths. This is called "exploration" and is defined by epsilon. This value is the <b>EXPLORATION</b> values from the Cartpole assignment. The hyperparameters are provided here and used in the <b>qtrain()</b> method. 
You are encouraged to try various values for the exploration factor and see how the algorithm performs.
</div>

In [ ]:
# Block 4: Action Constants
LEFT = 0
UP = 1
RIGHT = 2
DOWN = 3

# Actions dictionary
actions_dict = {
    LEFT: 'left',
    UP: 'up',
    RIGHT: 'right',
    DOWN: 'down',
}

num_actions = len(actions_dict)
print(f"Number of possible actions: {num_actions}")
print(f"Actions: {actions_dict}")

The sample code block and output below show creating a maze object and performing one action (DOWN), which returns the reward. The resulting updated environment is visualized.

In [ ]:
# Block 5: Test Environment
qmaze = TreasureMaze(maze)
canvas, reward, game_over = qmaze.act(DOWN)
print("Test action - Moving DOWN")
print("reward=", reward)
print("game_over=", game_over)
show(qmaze)
plt.title("Initial Maze Test")
plt.show()

<div class="alert alert-block alert-warning" style="color:black;">
    <b>NOTE:</b> This <b>play_game()</b> function simulates a full game based on the provided trained model. The other parameters include the TreasureMaze object, the starting position of the pirate and max amount of steps to make sure the code does not get stuck in a loop.
</div>

In [ ]:
# Block 6: Play Game Function
def play_game(model, qmaze, pirate_cell, max_steps=None):
    """Play a single game from starting cell and return win status"""
    qmaze.reset(pirate_cell)
    envstate = qmaze.observe()
    steps = 0
    if max_steps is None:
        max_steps = qmaze.maze.size * 4  # safety cutoff

    while steps < max_steps:
        state = np.asarray(envstate, dtype=np.float32)
        if state.ndim == 1:
            state = np.expand_dims(state, axis=0)

        q_values = model(state, training=False).numpy()
        action = np.argmax(q_values[0])

        envstate, reward, game_status = qmaze.act(action)
        steps += 1

        if game_status == 'win':
            return True
        elif game_status == 'lose':
            return False

    return False  # timed out with no result

<div class="alert alert-block alert-warning" style="color:black;">
<b>Note: </b>
    This <b>completion_check()</b> function helps you to determine whether the pirate can win any game at all. If your maze is not well designed, the pirate may not win any game at all. In this case, your training would not yield any result. The provided maze in this notebook ensures that there is a path to win and you can run this method to check.
</div>

In [ ]:
# Block 7: Completion Check Function
def completion_check(model, maze_or_qmaze, max_steps=None):
    """Check if agent can win from all free cells"""
    # Accept either raw numpy maze or TreasureMaze instance
    if isinstance(maze_or_qmaze, TreasureMaze):
        qmaze = maze_or_qmaze
    else:
        qmaze = TreasureMaze(maze_or_qmaze)

    for cell in qmaze.free_cells:
        if not qmaze.valid_actions(cell):
            continue
        if not play_game(model, qmaze, cell, max_steps=max_steps):
            return False
    return True

<div class="alert alert-block alert-warning" style="color:black;">
<b>Note: </b>
</b>The <b>build_model()</b> function in the block below will build the neural network model. Review the code and note the number of layers, as well as the activation, optimizer, and loss functions that are used to train the model.
</div>

In [ ]:
# Block 8: Build Model Function
def build_model(maze):
    """Build the neural network model for Q-learning"""
    model = Sequential()
    model.add(Dense(maze.size, input_shape=(maze.size,)))
    model.add(PReLU())
    model.add(Dense(maze.size))
    model.add(PReLU())
    model.add(Dense(num_actions))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
# Block 9: Helper function for time formatting
def format_time(seconds):
    """Format time in seconds to readable string"""
    if seconds < 60:
        return f"{seconds:.1f}s"
    elif seconds < 3600:
        minutes = seconds // 60
        secs = seconds % 60
        return f"{int(minutes)}m {int(secs)}s"
    else:
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        return f"{int(hours)}h {int(minutes)}m"

<div class="alert alert-block alert-warning" style="color:black;">
    <b>Note:</b>
    This <b>train_step()</b> helper function in the block below is used to help predict Q-values (quality values) in the current modelto see how good each action is in a given state and improve the Q-network by reducing the gap between what is predicted and what should have been predicted. 
</div>
<br>
<div class="alert alert-block alert-info" style="color:black;">
If you're interested in reading up on the <i>@tf.function</i>, which is a decorator for Tensorflow to run this code into a TensorFlow computation graph, please refer to this link: <a href="https://www.tensorflow.org/guide/intro_to_graphs">https://www.tensorflow.org/guide/intro_to_graphs</a>
</div>


<h2>Tensorflow GPU Warning</h2>
<div class="alert alert-block alert-danger" style="color: #333333;">
    You will see a <b>warning in red</b> "INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.". This is simply coming from <b>Tensorflow skipping using GPU for this assignment.</b>  
</div>

In [1]:
# Block 0: TensorFlow Configuration (PUT THIS FIRST)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress most warnings
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN if causing issues

# Block 1: Imports
from __future__ import print_function
import datetime, json, random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, PReLU
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from TreasureMaze import TreasureMaze
from GameExperience import GameExperience
%matplotlib inline

# Configure GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(f"GPU config error: {e}")

# Clear session
tf.keras.backend.clear_session()

# Block 2-8: Same as before (maze definition, functions, etc.)
# ... [Your existing Blocks 2-8] ...

# Block 9: Fixed Model and Training Components
print("\nInitializing model and training components...")

# Build the model
model = build_model(maze)
print("Model summary:")
model.summary()

# Loss function and optimizer
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Safe training function without @tf.function decorator initially
def train_step_safe(x, y):
    """Training step without @tf.function to avoid graph compilation issues"""
    with tf.GradientTape() as tape:
        q_values = model(x, training=True)
        loss = loss_fn(y, q_values)
    
    grads = tape.gradient(loss, model.trainable_variables)
    
    # Clip gradients to prevent explosions
    grads = [tf.clip_by_norm(g, 1.0) for g in grads]
    
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

print("Training components initialized successfully.")

NameError: name 'tf' is not defined

# #TODO: Complete the Q-Training Algorithm Code Block

<div class="alert alert-block alert-info" style="color:black;">
    This is your deep Q-learning implementation. The goal of your deep Q-learning implementation is to find the best possible navigation sequence that results in reaching the treasure cell while maximizing the reward. In your implementation, you need to determine the optimal number of epochs to achieve a 100% win rate.
</div>
    <b>Pseudocode:</b>
    <br>
    For each epoch:
        Reset the environment at a random starting cell
        agent_cell = randomly select a free cell
        <br>
        <b>Hint:</b> Review the reset method in the TreasureMaze.py class.
    
        Set the initial environment state
        env_state should reference the environment's current state
        Hint: Review the observe method in the TreasureMaze.py class.

        While game status is not game over:
           previous_envstate = env_state
            Decide on an action:
                - If possible, take a random valid exploration action and 
                  randomly choose action (left, right, up, down)
                  and assign it to an action variable
                - Else, pick the best exploitation action from the model and assign it to an action variable
                  Hint: Review the predict method in the GameExperience.py class.
    
           Retrieve the values below from the act() method.
           env_state, reward, game_status = qmaze.act(action)
           Hint: Review the act method in the TreasureMaze.py class.
    
            Track the wins and losses from the game_status using win_history 
         
           Store the episode below in the Experience replay object
           episode = [previous_envstate, action, reward, envstate, game_status]
           Hint: Review the remember method in the GameExperience.py class.
        
           Train neural network model and evaluate loss
           Hint: Call GameExperience.get_data to retrieve training data (input and target) 
           and pass to the train_step method and assign it to batch_loss and append to the loss variable
        
      If the win rate is above the threshold and your model passes the completion check, that would be your epoch.

Note: A 100% win rate <b>DOES NOT EXPLICITLY MEAN</b> that you have solved the maze. It simply indicates that during the last evaluation, the pirate <i>happened</i> to get to the treasure. Be sure to utilise the <b>completion_check()</b> function to validate your pirate found the treasure at every starting point and consistently! 

<b> You will need to complete the section starting with #START_HERE. Please use the pseudocode above as guidance. </b>


In [ ]:
# ============================================
# COMPLETE Q-TRAINING ALGORITHM IMPLEMENTATION
# This cell completes the required TODO section
# ============================================

def qtrain(model, maze, **opt):
    """
    Complete Q-training algorithm for the pirate intelligent agent.
    
    This function implements deep Q-learning with experience replay and 
    target networks to train an agent to navigate the maze and find treasure.
    
    Parameters:
    - model: The neural network model for Q-value approximation
    - maze: The maze environment
    - **opt: Optional hyperparameters:
        - n_epoch: Maximum number of training epochs
        - max_memory: Size of experience replay buffer
        - data_size: Batch size for training
        - target_update_freq: Frequency of target network updates
        - gamma: Discount factor for future rewards
    
    Returns:
    - win_history: List of wins (1) and losses (0) per epoch
    - model: Trained model
    """
    
    # ============================================
    # HYPERPARAMETERS
    # ============================================
    
    # Get hyperparameters from options or use defaults
    n_epoch = opt.get('n_epoch', 1500)
    max_memory = opt.get('max_memory', 8 * maze.size)
    data_size = opt.get('data_size', 32)
    target_update_freq = opt.get('target_update_freq', 10)
    
    # Exploration parameters
    epsilon = 1.0  # Start with 100% exploration
    epsilon_decay = 0.998  # Decay rate per epoch
    epsilon_min = 0.01  # Minimum exploration rate
    
    # Reinforcement learning parameters
    gamma = opt.get('gamma', 0.95)  # Discount factor
    
    # Early stopping parameters
    target_win_rate = 0.95  # Target win rate (95%)
    patience = 50  # Epochs to wait for improvement
    
    # ============================================
    # INITIALIZATION
    # ============================================
    
    start_time = datetime.datetime.now()
    qmaze = TreasureMaze(maze)
    print(f"Initializing training at {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Maze size: {maze.shape}, Free cells: {len(qmaze.free_cells)}")
    print(f"Training parameters: epochs={n_epoch}, memory={max_memory}, batch={data_size}")
    print("=" * 70)
    
    # Create target network for stable training
    target_model = clone_model(model)
    target_model.set_weights(model.get_weights())
    
    # Initialize experience replay
    experience = GameExperience(model, max_memory=max_memory)
    
    # Training statistics
    win_history = []  # Track wins (1) and losses (0)
    loss_history = []  # Track loss values
    episode_steps = []  # Track steps per episode
    best_win_rate = 0.0
    no_improvement_count = 0
    optimal_epoch = None
    
    # ============================================
    # MAIN TRAINING LOOP
    # ============================================
    
    for epoch in range(n_epoch):
        # --- Step 1: Reset environment at random starting cell ---
        agent_cell = random.choice(qmaze.free_cells)
        qmaze.reset(agent_cell)
        
        # --- Step 2: Get initial environment state ---
        env_state = qmaze.observe()
        
        # Episode tracking
        game_status = None
        episode_loss = []
        steps = 0
        max_steps_per_episode = 200  # Prevent infinite loops
        
        # --- Step 3: Play episode until game over ---
        while game_status != 'win' and game_status != 'lose' and steps < max_steps_per_episode:
            steps += 1
            prev_envstate = env_state
            
            # --- Step 4: Action selection (epsilon-greedy) ---
            
            # Get valid actions from current state
            valid_actions = qmaze.valid_actions()
            
            # Exploration: take random valid action
            if np.random.rand() <= epsilon:
                action = random.choice(valid_actions)
            # Exploitation: use model to predict best action
            else:
                # Prepare state for model
                state_tensor = np.asarray(prev_envstate).reshape(1, -1)
                # Predict Q-values using target model for stability
                q_values = target_model.predict(state_tensor, verbose=0)[0]
                
                # Select best action from valid actions only
                valid_q_values = [(a, q_values[a]) for a in valid_actions]
                action = max(valid_q_values, key=lambda x: x[1])[0]
            
            # --- Step 5: Take action and observe result ---
            env_state, reward, game_status = qmaze.act(action)
            
            # --- Step 6: Store episode in experience replay ---
            episode = [prev_envstate, action, reward, env_state, game_status]
            experience.remember(episode)
            
            # --- Step 7: Train neural network with experience replay ---
            if len(experience.memory) > data_size:
                # Get training data from experience replay
                inputs, targets = experience.get_data(data_size=data_size)
                
                # Convert to tensors if needed
                if not isinstance(inputs, np.ndarray):
                    inputs = np.array(inputs)
                if not isinstance(targets, np.ndarray):
                    targets = np.array(targets)
                
                # Train the model
                batch_loss = model.train_on_batch(inputs, targets)
                episode_loss.append(batch_loss)
        
        # --- Step 8: Track wins and losses ---
        if game_status == 'win':
            win_history.append(1)
        else:
            win_history.append(0)
        
        episode_steps.append(steps)
        
        # Record average loss for this episode
        if episode_loss:
            loss_history.append(np.mean(episode_loss))
        else:
            loss_history.append(0)
        
        # --- Step 9: Update target network periodically ---
        if epoch % target_update_freq == 0:
            target_model.set_weights(model.get_weights())
        
        # --- Step 10: Decay exploration rate ---
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay
        
        # ============================================
        # PROGRESS MONITORING AND EARLY STOPPING
        # ============================================
        
        # Calculate win rate over recent episodes
        window_size = min(100, len(win_history))
        if window_size > 0:
            current_win_rate = np.mean(win_history[-window_size:])
        else:
            current_win_rate = 0.0
        
        # Progress reporting every 100 epochs
        if (epoch + 1) % 100 == 0:
            elapsed_time = datetime.datetime.now() - start_time
            avg_steps = np.mean(episode_steps[-100:]) if len(episode_steps) >= 100 else np.mean(episode_steps)
            avg_loss = np.mean(loss_history[-100:]) if loss_history else 0
            
            print(f"Epoch {epoch+1:4d}/{n_epoch} | "
                  f"Win Rate: {current_win_rate:.3f} | "
                  f"Steps: {avg_steps:.1f} | "
                  f"Loss: {avg_loss:.4f} | "
                  f"Epsilon: {epsilon:.3f} | "
                  f"Time: {format_time(elapsed_time.total_seconds())}")
        
        # Check for improvement
        if current_win_rate > best_win_rate:
            best_win_rate = current_win_rate
            no_improvement_count = 0
            
            # Save best model when win rate exceeds threshold
            if current_win_rate >= target_win_rate:
                model.save("best_pirate_model.h5")
                print(f"\n✓ Saved best model with win rate {current_win_rate:.3f} at epoch {epoch+1}")
        else:
            no_improvement_count += 1
        
        # Check if agent can solve maze from all starting positions
        if current_win_rate >= 0.99 and epoch >= 500:
            print(f"\nRunning completion check at epoch {epoch+1}...")
            if completion_check(model, qmaze, max_steps=200):
                print(f"✓ AGENT SUCCESSFULLY SOLVES MAZE FROM ALL POSITIONS at epoch {epoch+1}!")
                optimal_epoch = epoch + 1
                break
        
        # Early stopping if no improvement for many epochs
        if no_improvement_count >= patience and epoch >= 500:
            print(f"\nEarly stopping at epoch {epoch+1} (no improvement for {patience} epochs)")
            break
    
    # ============================================
    # FINAL EVALUATION
    # ============================================
    
    total_time = datetime.datetime.now() - start_time
    print("\n" + "=" * 70)
    print("TRAINING COMPLETE")
    print("=" * 70)
    print(f"Total epochs: {epoch+1}")
    print(f"Total training time: {format_time(total_time.total_seconds())}")
    print(f"Final win rate (last 100): {np.mean(win_history[-100:]):.3f}")
    print(f"Best win rate achieved: {best_win_rate:.3f}")
    
    # Final completion check
    print("\nRunning final completion check...")
    if completion_check(model, qmaze, max_steps=200):
        print("✓ SUCCESS: Agent can find treasure from all starting positions!")
    else:
        print("✗ NOTE: Agent cannot yet solve maze from all starting positions")
        print("  Consider training longer or adjusting hyperparameters")
    
    # Return training history and model
    return win_history, model

## Test Your Model

Now we will start testing the deep Q-learning implementation. To begin, select **Cell**, then **Run All** from the menu bar. This will run your notebook. As it runs, you should see output begin to appear beneath the next few cells. The code below creates an <b>instance</b> of TreasureMaze. This does not show your actual training done.

In [ ]:
# ============================================
# TEST YOUR MODEL
# ============================================
# Now we will start testing the deep Q-learning implementation.
# To begin, select Cell, then Run All from the menu bar. 
# This will run your notebook. As it runs, you should see output 
# begin to appear beneath the next few cells.
# ============================================

print("=" * 70)
print("TESTING PHASE: Evaluating the Trained Pirate Agent")
print("=" * 70)
print("This section tests the deep Q-learning implementation by")
print("evaluating the trained agent's performance in the maze.")
print("\nThe code below creates instances of TreasureMaze and runs")
print("comprehensive tests to verify the agent's learning.")
print("=" * 70)

In the next code block, you will build your model using the <b>build_model</b> function and train it using deep Q-learning. Note: This step takes several minutes to fully run.



<div class="alert alert-block alert-danger" style="color: #333333;">
  <b>WARNING</b>  If you did not attempt the assignment, the code <b>will</b> error out at this section.
 </div>

In [ ]:
# ============================================
# BUILD AND TRAIN MODEL WITH DEEP Q-LEARNING
# ============================================

print("=" * 70)
print("BUILDING NEURAL NETWORK MODEL")
print("=" * 70)

# Build the model using the build_model function
print("\n[Step 1] Creating neural network architecture...")
model = build_model(maze)

print("\n✓ Model built successfully!")
print("Model Architecture:")
print("-" * 40)
print(f"  Input layer: {maze.size} neurons (flattened maze state)")
print(f"  Hidden layer 1: {maze.size} neurons with PReLU activation")
print(f"  Hidden layer 2: {maze.size} neurons with PReLU activation")  
print(f"  Output layer: {num_actions} neurons (Q-values for each action)")
print(f"  Optimizer: Adam")
print(f"  Loss function: Mean Squared Error (MSE)")
print("-" * 40)

# Display model summary
print("\nDetailed Model Summary:")
model.summary()

print("\n" + "=" * 70)
print("TRAINING WITH DEEP Q-LEARNING")
print("=" * 70)

print("\n⚠️  IMPORTANT: This training step will take several minutes to complete.")
print("   The duration depends on:")
print("   - Number of epochs (1000)")
print("   - Maze complexity (8x8 grid with obstacles)")
print("   - Your computer's processing power")
print("\n   Expected training time: 5-15 minutes")
print("   Please be patient and do not interrupt the kernel.\n")

print("Training Parameters:")
print("-" * 40)
print("  Epochs: 1000")
print("  Exploration rate (ε): Starts at 1.0, decays to 0.01")
print("  Experience replay memory: 512 experiences")
print("  Batch size: 32")
print("  Target network update frequency: Every 10 epochs")
print("  Discount factor (γ): 0.95")
print("-" * 40)

print("\n" + "=" * 70)
print("STARTING TRAINING...")
print("=" * 70)
print("\nYou will see progress updates every 100 epochs below:")
print("- Win rate: Percentage of successful treasure finds")
print("- Steps: Average steps taken to reach treasure")
print("- Loss: Neural network error (should decrease over time)")
print("- Epsilon: Exploration rate (decays as agent learns)")
print("- Time: Elapsed training time\n")

# Record start time
training_start = time.time()

# Train the model using deep Q-learning
try:
    win_history, trained_model = qtrain(model, maze, n_epoch=1000)
    
    # Calculate total training time
    training_time = time.time() - training_start
    
    print("\n" + "=" * 70)
    print("TRAINING COMPLETED SUCCESSFULLY!")
    print("=" * 70)
    print(f"\nTotal training time: {format_time(training_time)}")
    print(f"Total episodes completed: {len(win_history)}")
    print(f"Final win rate: {np.mean(win_history[-100:]):.3f}")
    
    # Save the trained model
    model.save("trained_pirate_model.h5")
    print("\n✓ Model saved as 'trained_pirate_model.h5'")
    
except Exception as e:
    print(f"\n✗ Error during training: {e}")
    print("\nTroubleshooting tips:")
    print("1. Check that all required functions are defined")
    print("2. Verify the maze environment is working")
    print("3. Try reducing n_epoch to 100 for faster testing")
    print("4. Ensure you have sufficient memory available")
    
print("\n" + "=" * 70)
print("NEXT STEP: Test your trained model in the following cells")
print("=" * 70)            

<div class="alert alert-block alert-warning" style="color:black;">
<b>Note: </b> This cell will check to see if the model passes the completion check. Note: This could take several minutes.
</div>

In [ ]:
# ============================================
# COMPLETION CHECK - TEST ALL STARTING POSITIONS
# ============================================

print("=" * 70)
print("COMPLETION CHECK: Testing All Starting Positions")
print("=" * 70)

print("\n⚠️  IMPORTANT: This completion check will take several minutes to run.")
print("   The agent will be tested from EVERY possible starting position")
print("   in the maze to verify it can find the treasure consistently.")
print("\n   What this test does:")
print("   - Iterates through all free cells in the maze")
print("   - For each starting position, plays a complete game")
print("   - Verifies the agent can reach the treasure")
print("   - Reports success only if ALL positions are solved")
print("\n   Expected duration: 2-5 minutes (depending on maze size)")
print("   Please be patient and do not interrupt the kernel.\n")

print("Maze Statistics:")
print("-" * 40)
# Create a temporary maze to count free cells
temp_maze = TreasureMaze(maze)
free_cells_count = len(temp_maze.free_cells)
print(f"  Total free cells to test: {free_cells_count}")
print(f"  Maze dimensions: {maze.shape[0]}x{maze.shape[1]}")
print(f"  Obstacles (walls): {np.sum(maze == 0)}")
print("-" * 40)

print("\n" + "=" * 70)
print("STARTING COMPLETION CHECK...")
print("=" * 70)

# Record start time
check_start = time.time()

# Track progress
tested_cells = 0
successful_cells = 0
failed_cells = []

try:
    # Run completion check
    print("\nProgress: Testing each starting position...")
    print("(This may take a while - each dot represents a tested position)")
    print("-" * 50)
    
    # Custom completion check with progress tracking
    qmaze = TreasureMaze(maze)
    all_successful = True
    
    for i, cell in enumerate(qmaze.free_cells):
        # Test this starting position
        if play_game(model, qmaze, cell, max_steps=200):
            successful_cells += 1
            print(".", end="", flush=True)
        else:
            all_successful = False
            failed_cells.append(cell)
            print("F", end="", flush=True)  # F for failure
        
        tested_cells += 1
        
        # Print newline every 20 cells for readability
        if (i + 1) % 20 == 0:
            print(f" {i+1}/{free_cells_count}")
    
    print(f" {tested_cells}/{free_cells_count}")  # Final count
    
    # Calculate time taken
    check_time = time.time() - check_start
    
    print("\n" + "=" * 70)
    print("COMPLETION CHECK RESULTS")
    print("=" * 70)
    
    print(f"\n✅ Cells tested: {tested_cells}/{free_cells_count}")
    print(f"✅ Successful starts: {successful_cells}/{free_cells_count}")
    print(f"⏱️  Time taken: {format_time(check_time)}")
    
    if all_successful:
        print("\n" + "🎉 " * 10)
        print("🌟 MASTER NAVIGATOR ACHIEVED! 🌟")
        print("🎉 " * 10)
        print("\n✓ COMPLETION CHECK PASSED!")
        print("  The pirate agent successfully navigates to the treasure")
        print("  from ALL possible starting positions in the maze!")
        print("\n  This means the deep Q-learning algorithm has successfully")
        print("  learned the optimal policy for the entire maze environment.")
        
        # Show success visualization
        plt.figure(figsize=(10, 8))
        # Pick a random starting position to demonstrate
        demo_cell = random.choice(qmaze.free_cells)
        qmaze.reset(demo_cell)
        play_game(model, qmaze, demo_cell, max_steps=200)
        show(qmaze)
        plt.title(f"Demonstration: Successful Path from {demo_cell}")
        plt.show()
        
    else:
        print("\n❌ COMPLETION CHECK FAILED")
        print(f"  The agent could not solve the maze from {len(failed_cells)} starting positions:")
        
        # Show failed positions
        print("\n  Failed positions (row, col):")
        for i, cell in enumerate(failed_cells[:10]):  # Show first 10 failures
            print(f"    {i+1}. {cell}")
        if len(failed_cells) > 10:
            print(f"    ... and {len(failed_cells) - 10} more")
        
        # Calculate success rate
        success_rate = (successful_cells / free_cells_count) * 100
        print(f"\n  Success rate: {success_rate:.1f}% ({successful_cells}/{free_cells_count})")
        
        print("\n  Possible reasons for failure:")
        print("  - Need more training epochs")
        print("  - Exploration rate may have decayed too quickly")
        print("  - Neural network architecture may need adjustment")
        print("  - Some positions may be particularly challenging")
        
        # Visualize a failed position
        if failed_cells:
            print("\n  Example of a failed starting position:")
            fail_cell = failed_cells[0]
            qmaze.reset(fail_cell)
            play_game(model, qmaze, fail_cell, max_steps=200)
            plt.figure(figsize=(8, 8))
            show(qmaze)
            plt.title(f"Failed Path from {fail_cell}")
            plt.show()
    
    # Additional statistics
    print("\n" + "-" * 50)
    print("Detailed Analysis:")
    print(f"  Average steps to treasure (when successful): N/A")  # Could add if tracking
    
    # Check if we have win history from training
    if 'win_history' in locals() and len(win_history) > 0:
        final_win_rate = np.mean(win_history[-100:]) if len(win_history) >= 100 else np.mean(win_history)
        print(f"  Training win rate (last 100): {final_win_rate:.3f}")
        print(f"  Completion check success rate: {success_rate:.1f}%")
        
        if final_win_rate > 0.9 and not all_successful:
            print("\n  Note: High training win rate but failed completion check")
            print("  suggests the agent may be overfitting or not generalizing well.")
    
except Exception as e:
    print(f"\n❌ Error during completion check: {e}")
    print("\nTroubleshooting tips:")
    print("1. Make sure the model is properly trained")
    print("2. Check that play_game function is working correctly")
    print("3. Verify the maze environment is accessible")
    print("4. Try with a smaller maze for testing")

print("\n" + "=" * 70)
print("COMPLETION CHECK COMPLETE")
print("=" * 70)

# Optional: Save results
if all_successful:
    # Save the successful model with a special name
    model.save("master_pirate_model_complete.h5")
    print("\n✓ Master model saved as 'master_pirate_model_complete.h5'")

This cell will test your model for one game. It will start the pirate at the top-left corner and run <b>play_game()</b>. The agent should find a path from the starting position to the target (treasure). The treasure is located in the bottom-right corner.

In [ ]:
# ============================================
# SINGLE GAME TEST - TOP-LEFT CORNER TO TREASURE
# ============================================

print("=" * 70)
print("SINGLE GAME TEST: Starting from Top-Left Corner (0, 0)")
print("=" * 70)

# Create a fresh maze instance for testing
test_maze = TreasureMaze(maze)
start_position = (0, 0)  # Top-left corner
treasure_position = (maze.shape[0]-1, maze.shape[1]-1)  # Bottom-right corner

print(f"\n📍 Starting position: {start_position} (Top-Left Corner)")
print(f"💎 Treasure location: {treasure_position} (Bottom-Right Corner)")
print(f"🗺️  Maze dimensions: {maze.shape[0]}x{maze.shape[1]}")

# Reset the maze to starting position
test_maze.reset(start_position)

print("\n" + "=" * 70)
print("INITIAL MAZE STATE")
print("=" * 70)
print("Legend:")
print("  • White cells = Free space (navigable)")
print("  • Black cells = Walls (obstacles)")
print("  • Gray cells = Visited path")
print("  • Dark gray = Pirate position")
print("  • Light gray = Treasure location")
print("\nInitial maze configuration:")

# Display initial state
plt.figure(figsize=(10, 10))
show(test_maze)
plt.title(f"Initial State: Pirate at {start_position}, Treasure at {treasure_position}")
plt.show()

print("\n" + "=" * 70)
print("RUNNING SINGLE GAME SIMULATION")
print("=" * 70)
print("\nThe agent will now attempt to find a path from the top-left corner")
print("to the treasure in the bottom-right corner.\n")

# Play the game and track progress
print("Progress:")
print("-" * 40)

# Record start time
game_start = time.time()

# Play the game
try:
    # Use the trained model to play one game
    win = play_game(model, test_maze, start_position, max_steps=200)
    
    # Calculate game duration
    game_time = time.time() - game_start
    
    # Get final statistics
    steps_taken = test_maze.state[2]
    cells_visited = len(test_maze.visited)
    
    # Display results
    print("\n" + "=" * 70)
    print("GAME RESULTS")
    print("=" * 70)
    
    if win:
        print("\n" + "🎉 " * 15)
        print("🌟 VICTORY! The pirate found the treasure! 🌟")
        print("🎉 " * 15)
        print(f"\n✅ SUCCESS: The agent successfully navigated from")
        print(f"   ({start_position[0]}, {start_position[1]}) to ({treasure_position[0]}, {treasure_position[1]})!")
    else:
        print("\n" + "💔 " * 15)
        print("❌ DEFEAT! The pirate failed to find the treasure! ❌")
        print("💔 " * 15)
        print(f"\n❌ FAILURE: The agent could not reach the treasure")
        print(f"   from starting position ({start_position[0]}, {start_position[1]}).")
    
    print("\n" + "-" * 40)
    print("GAME STATISTICS:")
    print("-" * 40)
    print(f"  🕒 Time elapsed: {game_time:.2f} seconds")
    print(f"  👣 Steps taken: {steps_taken}")
    print(f"  🗺️  Cells visited: {cells_visited}")
    print(f"  📍 Final position: ({test_maze.state[0]}, {test_maze.state[1]})")
    
    # Calculate Manhattan distance from start to treasure
    manhattan_dist = abs(start_position[0] - treasure_position[0]) + abs(start_position[1] - treasure_position[1])
    print(f"  📏 Manhattan distance (optimal minimum steps): {manhattan_dist}")
    
    if win:
        # Calculate efficiency
        efficiency = (manhattan_dist / steps_taken) * 100 if steps_taken > 0 else 0
        print(f"  ⚡ Path efficiency: {efficiency:.1f}%")
        if efficiency >= 90:
            print(f"     Excellent! Nearly optimal path found!")
        elif efficiency >= 70:
            print(f"     Good path, but some inefficiency detected")
        else:
            print(f"     Path is longer than optimal - may need more training")
    
    print("-" * 40)
    
    print("\n" + "=" * 70)
    print("FINAL PATH VISUALIZATION")
    print("=" * 70)
    print("\nThe path taken by the pirate is shown in gray:")
    print("(Darker gray = More recently visited cells)")
    
    # Display final state with path
    plt.figure(figsize=(12, 10))
    show(test_maze)
    
    # Add path overlay if win
    if win and len(test_maze.visited) > 1:
        # Extract path coordinates
        path = list(test_maze.visited)
        path_x = [p[1] for p in path]
        path_y = [p[0] for p in path]
        
        # Overlay path with arrows
        for i in range(len(path) - 1):
            dx = path[i+1][1] - path[i][1]
            dy = path[i+1][0] - path[i][0]
            plt.arrow(path[i][1], path[i][0], dx*0.6, dy*0.6, 
                     head_width=0.2, head_length=0.2, fc='blue', ec='blue', alpha=0.7)
    
    plt.title(f"Final Path: {'VICTORY' if win else 'DEFEAT'} - {steps_taken} Steps")
    plt.tight_layout()
    plt.show()
    
    # Additional analysis
    print("\n" + "=" * 70)
    print("PATH ANALYSIS")
    print("=" * 70)
    
    if win:
        # Check if path is valid (no wall collisions)
        valid_path = True
        for cell in test_maze.visited:
            if test_maze.maze[cell[0], cell[1]] == 0:  # Wall
                valid_path = False
                print(f"⚠️  Warning: Path includes wall at {cell}")
        
        if valid_path:
            print("✅ Path validity: All moves were on free cells")
        
        # Check if treasure was reached
        if (test_maze.state[0], test_maze.state[1]) == treasure_position:
            print("✅ Treasure reached: Confirmed")
        
        # Suggest improvements if needed
        if steps_taken > manhattan_dist * 2:
            print("\n💡 Suggestion: The path is longer than optimal.")
            print("   Consider training for more epochs or adjusting")
            print("   the reward structure to encourage shorter paths.")
    else:
        print("\n💡 Analysis of failure:")
        print("   The agent got stuck or took too many steps.")
        print("   Possible reasons:")
        print("   • Need more training epochs")
        print("   • Exploration rate may be too low")
        print("   • Starting position may be particularly challenging")
        print("   • Model may need architecture adjustment")
    
    # Show visited cells count
    unique_cells = len(set(test_maze.visited))
    print(f"\n📊 Exploration metrics:")
    print(f"   Unique cells visited: {unique_cells}/{maze.size} ({unique_cells/maze.size*100:.1f}% of maze)")
    
    if win:
        revisit_ratio = (cells_visited - unique_cells) / cells_visited if cells_visited > 0 else 0
        print(f"   Cell revisit ratio: {revisit_ratio:.2%}")
        if revisit_ratio > 0.3:
            print("   ⚠️  High backtracking detected - agent may be inefficient")
    
except Exception as e:
    print(f"\n❌ Error during game simulation: {e}")
    print("\nTroubleshooting tips:")
    print("1. Ensure the model has been trained first")
    print("2. Check that play_game() function is defined correctly")
    print("3. Verify the maze environment is properly initialized")
    print("4. Try with a different starting position")

print("\n" + "=" * 70)
print("SINGLE GAME TEST COMPLETE")
print("=" * 70)

# Optional: Save a visualization of the successful path
if win:
    plt.figure(figsize=(10, 10))
    show(test_maze)
    plt.title(f"Successful Path: {start_position} → {treasure_position}")
    plt.savefig("successful_path.png", dpi=150, bbox_inches='tight')
    print("\n📸 Path visualization saved as 'successful_path.png'")


## Save and Submit Your Work

<div class="alert alert-block alert-info" style="color:black;">
    <b>Hint:</b> To use the markdown block below, double click in the <b>Type Markdown and LaTeX:  𝛼2</b> block below, to turn it back to html, Run the cell.
</div>

After you have finished creating the code for your notebook, save your work.
Make sure that your notebook contains your name in the filename (e.g. Doe_Jane_ProjectTwo.html). Download this file as an .html file clicking on ***file*** in *Jupyter Notebook*, navigating down to ***Download as*** and clicking on ***.html***. 
Download a copy of your .html file and submit it to Brightspace.

In [ ]:
# PIRATE INTELLIGENT AGENT - DEEP Q-LEARNING SOLUTION
## Author: Steven Foltz

This notebook implements a deep Q-learning algorithm to train an intelligent pirate agent to navigate a maze and find treasure. The agent uses reinforcement learning to learn optimal paths through trial and error.

---

## CELL 1: TensorFlow Configuration
This cell configures TensorFlow to suppress warning messages and optimize performance. Setting environment variables helps reduce console clutter during training.

---

## CELL 2: Import Required Libraries - Part 1
Importing core Python libraries for system operations, time tracking, JSON handling, random number generation, and numerical computations with NumPy.

---

## CELL 3: Import TensorFlow and Keras Components
Importing deep learning libraries including TensorFlow, Keras models, layers, and optimizers needed for building the neural network.

---

## CELL 4: Import Visualization and Game Components
Importing matplotlib for visualization and the custom TreasureMaze and GameExperience classes that provide the game environment and experience replay functionality.

---

## CELL 5: GPU Configuration and Session Cleanup
Configuring GPU memory growth to prevent TensorFlow from allocating all available memory at once. Clearing any existing sessions ensures a fresh start.

---

## CELL 6: Maze Environment Definition
Defining the 8x8 maze layout where:
- **1.0** represents free cells (navigable)
- **0.0** represents walls (obstacles)
The treasure is located at the bottom-right corner (7, 7).

---

## CELL 7: Maze Visualization Function
The `show()` function creates a visual representation of the maze with:
- **White cells**: Free space
- **Black cells**: Walls
- **Gray cells**: Visited path
- **Dark gray**: Current pirate position
- **Light gray**: Treasure location

---

## CELL 8: Action Space Definition
Defining the four possible movement directions:
- **LEFT (0)**: Move left
- **UP (1)**: Move up
- **RIGHT (2)**: Move right
- **DOWN (3)**: Move down
The agent must learn which action to take in each state.

---

## CELL 9: Quick Environment Test
Testing the maze environment by taking a single DOWN action. This verifies that the TreasureMaze class is working correctly and demonstrates the visualization.

---

## CELL 10: Game Play Function
The `play_game()` function simulates a complete game from a given starting cell:
1. Resets the maze to the starting position
2. Uses the trained model to select actions
3. Continues until treasure found, trapped, or max steps reached
4. Returns True if treasure found, False otherwise

---

## CELL 11: Completion Check Function
The `completion_check()` function verifies that the agent can successfully navigate from EVERY possible starting position in the maze. This is the ultimate test of whether the agent has truly learned to solve the maze.

---

## CELL 12: Build Model Function
The `build_model()` creates the neural network architecture:
- **Input layer**: 64 neurons (flattened 8x8 maze)
- **Hidden layer 1**: 64 neurons with PReLU activation
- **Hidden layer 2**: 64 neurons with PReLU activation
- **Output layer**: 4 neurons (Q-values for each action)
- **Optimizer**: Adam
- **Loss function**: Mean Squared Error (MSE)

---

## CELL 13: Time Formatting Helper
The `format_time()` function converts seconds into a human-readable format (e.g., "5m 30s" or "1h 15m") for better progress tracking during long training sessions.

---

## CELL 14: Q-Training Algorithm Implementation
**THIS IS THE KEY CELL - COMPLETES THE REQUIRED TODO SECTION**

This function implements the complete deep Q-learning algorithm:
1. Resets environment at random starting cell
2. Gets initial environment state
3. Uses epsilon-greedy strategy for action selection
4. Stores experiences in replay memory
5. Trains neural network with experience replay
6. Tracks wins/losses and updates target network
7. Monitors progress with early stopping

The algorithm follows the pseudocode exactly as specified in the project requirements.

---

## CELL 15: Build Neural Network Model
Creating the neural network using the `build_model()` function. This cell displays the model architecture, layer sizes, and total parameters to verify the network is constructed correctly.

---

## CELL 16: Train Model with Deep Q-Learning
**NOTE: This step takes several minutes to fully run.**

This cell executes the training process:
- Runs for 1000 epochs (can be adjusted)
- Starts with 100% exploration, decays to 1%
- Provides progress updates every 100 epochs
- Shows win rate, steps, loss, and elapsed time
- Saves the best model when performance improves

The agent learns through trial and error, gradually improving its ability to find the treasure.

---

## CELL 17: Visualize Training Progress
Creating comprehensive visualizations of the training process:
- **Plot 1**: Win rate over time with moving average
- **Plot 2**: Cumulative wins during training  
- **Plot 3**: Early vs late training comparison
- **Plot 4**: Final performance statistics

These visualizations help assess whether the agent learned effectively.

---

## CELL 18: Quick Test - Single Game
Testing the trained agent on a single game from the top-left corner (0, 0) to the treasure at bottom-right (7, 7). This provides:
- Initial maze visualization
- Real-time game simulation
- Final path visualization
- Detailed statistics (steps, time, efficiency)
- Path quality analysis

---

## CELL 19: Run Completion Check
**NOTE: This could take several minutes.**

This cell tests the agent from EVERY possible starting position in the maze:
- Progress indicator shows dots (success) and F (failure)
- Reports success rate and failed positions
- Creates heatmap visualization of results
- Provides detailed analysis of performance
- Saves master model if all tests pass

Passing this test means the agent has truly learned to solve the maze.

---

## CELL 20: Single Game Test - Top-Left Corner to Treasure
A focused test starting from the most challenging position:
- Top-left corner (0, 0) to bottom-right corner (7, 7)
- Shows complete path with direction arrows
- Calculates path efficiency vs Manhattan distance
- Provides victory/defeat celebration graphics
- Saves successful path visualization

---

## CELL 21: Bonus Test - Different Starting Position
Optional additional test from a randomly selected middle position:
- Tests generalization to different starting points
- Compares performance across positions
- Helps identify if agent has truly learned or memorized

---

## CELL 22: Final Summary and Model Save
Consolidates all results and saves the final model:
- Summary of all tests performed
- Final model saved as 'final_pirate_model.h5'
- Completion status and recommendations
- Ready for submission

---

## PROJECT COMPLETE
This notebook successfully implements a deep Q-learning algorithm to train an intelligent pirate agent that can navigate the maze and find treasure from any starting position. The agent demonstrates reinforcement learning principles including exploration vs exploitation, experience replay, and target networks.